# Generate and Push Manifests for All MMAP Data

This notebook scans all memory-mapped data on disk, generates manifest files for each cluster, and uploads them to the HPC cluster.

## What are Manifests?

Manifest files are JSON documents that describe the contents of each cluster:
- Cluster metadata (ID, country, dates)
- Complete tile inventory (coordinates, years)
- Generation timestamps

HPC systems use manifests to discover available data without needing a database.

## Workflow

1. Scan `data/landsat_mmap/` to find all cluster/year combinations
2. For each cluster, generate a manifest using database information
3. Save manifests locally to `data/manifests/`
4. Upload manifests to HPC at `$HPC_ZARR_PATH/manifests/`

In [1]:
import sys
import logging
from pathlib import Path
from collections import defaultdict

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent.parent))

from src.data.database import DownloadDatabase
from src.data.config import Config
from src.data.transfer import TransferWorker

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ Imports loaded")

✓ Imports loaded


## Step 1: Initialize Configuration and Database

In [3]:
# Initialize
config = Config()
db = DownloadDatabase(str(config.DB_PATH))
worker = TransferWorker(db, config)

print(f"Database: {config.DB_PATH}")
print(f"MMAP directory: {config.DATA_DIR / 'landsat_mmap'}")
print(f"Manifests directory: {worker.manifests_dir}")
print(f"HPC manifests path: {config.HPC_ZARR_PATH}/manifests")

2026-02-06 17:10:24,333 - INFO - Initialized transfer worker
2026-02-06 17:10:24,333 - INFO - HPC host: transfer12.scicore.unibas.ch
2026-02-06 17:10:24,333 - INFO - HPC user: schulz0022
2026-02-06 17:10:24,334 - INFO - HPC path: /scicore/home/meiera/schulz0022/projects/mining-net/data/landsat_mmap


Database: C:\Users\schulz0022\Documents\mining-net\data\mining_segmentation.db
MMAP directory: C:\Users\schulz0022\Documents\mining-net\data\landsat_mmap
Manifests directory: C:\Users\schulz0022\Documents\mining-net\data\manifests
HPC manifests path: /scicore/home/meiera/schulz0022/projects/mining-net/data/landsat_mmap/manifests


## Step 2: Scan MMAP Directory for Clusters

In [6]:
mmap_path = config.DATA_DIR / "landsat_mmap"

if not mmap_path.exists():
    print("❌ MMAP directory not found!")
    raise FileNotFoundError(f"MMAP directory does not exist: {mmap_path}")

# Scan for clusters
clusters_on_disk = defaultdict(set)

for cluster_dir in mmap_path.iterdir():
    if cluster_dir.is_dir():
        try:
            cluster_id = int(cluster_dir.name)
            
            # Check for year subdirectories
            for year_dir in cluster_dir.iterdir():
                if year_dir.is_dir():
                    try:
                        year = int(year_dir.name)
                        # Verify this year has actual tile data
                        has_tiles = any(tile_dir.is_dir() for tile_dir in year_dir.iterdir())
                        if has_tiles:
                            clusters_on_disk[cluster_id].add(year)
                    except ValueError:
                        pass
        except ValueError:
            pass

print(f"\n📊 Found {len(clusters_on_disk)} clusters on disk:")
print(f"   Total cluster/year combinations: {sum(len(years) for years in clusters_on_disk.values())}")

# Show summary
print("\n📋 Cluster summary:")
for cluster_id in sorted(clusters_on_disk.keys())[:10]:
    years = sorted(clusters_on_disk[cluster_id])
    print(f"   Cluster {cluster_id}: {len(years)} years ({min(years)}-{max(years)})")
if len(clusters_on_disk) > 10:
    print(f"   ... and {len(clusters_on_disk) - 10} more clusters")


📊 Found 228 clusters on disk:
   Total cluster/year combinations: 228

📋 Cluster summary:
   Cluster 6155687546600015: 1 years (2019-2019)
   Cluster 16714675725600725: 1 years (2019-2019)
   Cluster 17315396644879081: 1 years (2019-2019)
   Cluster 52652953960301590: 1 years (2019-2019)
   Cluster 62084203559214856: 1 years (2019-2019)
   Cluster 71137107360758801: 1 years (2019-2019)
   Cluster 77491409989930463: 1 years (2019-2019)
   Cluster 89022526743908725: 1 years (2019-2019)
   Cluster 101534883284040649: 1 years (2019-2019)
   Cluster 130239313423728009: 1 years (2019-2019)
   ... and 218 more clusters


## Step 3: Check Existing Manifests

In [7]:
# Find existing manifests
existing_manifests = set()
if worker.manifests_dir.exists():
    for manifest_file in worker.manifests_dir.glob("cluster_*_manifest.json"):
        try:
            # Extract cluster ID from filename
            cluster_id = int(manifest_file.stem.split('_')[1])
            existing_manifests.add(cluster_id)
        except (ValueError, IndexError):
            pass

print(f"\n📁 Existing manifests: {len(existing_manifests)}")

# Determine which clusters need manifests
clusters_needing_manifests = set(clusters_on_disk.keys()) - existing_manifests

print(f"🆕 Clusters needing manifests: {len(clusters_needing_manifests)}")
print(f"✓ Clusters with manifests: {len(existing_manifests)}")


📁 Existing manifests: 0
🆕 Clusters needing manifests: 228
✓ Clusters with manifests: 0


## Step 4: Generate Manifests

Generate manifest files for all clusters (or only missing ones).

In [8]:
# Configuration
REGENERATE_ALL = False  # Set to True to regenerate all manifests
DRY_RUN = False  # Set to True to preview without creating files

if REGENERATE_ALL:
    clusters_to_process = set(clusters_on_disk.keys())
    print(f"🔄 Regenerating ALL {len(clusters_to_process)} manifests...")
else:
    clusters_to_process = clusters_needing_manifests
    print(f"🆕 Generating {len(clusters_to_process)} new manifests...")

if DRY_RUN:
    print("\n⚠️  DRY RUN MODE - No files will be created\n")

# Generate manifests
successful = []
failed = []

for i, cluster_id in enumerate(sorted(clusters_to_process), 1):
    print(f"\r[{i}/{len(clusters_to_process)}] Processing cluster {cluster_id}...", end="", flush=True)
    
    if not DRY_RUN:
        try:
            manifest_path = worker._create_manifest(cluster_id)
            if manifest_path:
                successful.append(cluster_id)
            else:
                failed.append((cluster_id, "Manifest creation returned None"))
        except Exception as e:
            failed.append((cluster_id, str(e)))
    else:
        # Just check if cluster info exists in database
        cluster_info = db.get_cluster_info(cluster_id)
        if cluster_info:
            successful.append(cluster_id)
        else:
            failed.append((cluster_id, "No database info"))

print("\n")  # New line after progress

# Report results
print(f"\n{'=' * 60}")
print(f"📊 Manifest Generation Results")
print(f"{'=' * 60}")
print(f"✅ Successful: {len(successful)}")
print(f"❌ Failed: {len(failed)}")

if failed:
    print(f"\n⚠️  Failed clusters:")
    for cluster_id, error in failed[:10]:
        print(f"   Cluster {cluster_id}: {error}")
    if len(failed) > 10:
        print(f"   ... and {len(failed) - 10} more")

🆕 Generating 228 new manifests...
[1/228] Processing cluster 6155687546600015...

2026-02-06 17:11:14,510 - INFO - Created manifest for cluster 6155687546600015: 72 tiles across 1 years


[2/228] Processing cluster 16714675725600725...

2026-02-06 17:11:14,513 - INFO - Created manifest for cluster 16714675725600725: 169 tiles across 1 years


[3/228] Processing cluster 17315396644879081...

...

[227/228] Processing cluster 4573129916222891577...

2026-02-06 17:11:15,291 - INFO - Created manifest for cluster 4573129916222891577: 520 tiles across 1 years


[228/228] Processing cluster 4589502480873092105...

2026-02-06 17:11:15,294 - INFO - Created manifest for cluster 4589502480873092105: 72 tiles across 1 years





📊 Manifest Generation Results
✅ Successful: 228
❌ Failed: 0


## Step 5: Upload Manifests to HPC

Transfer all manifest files to the HPC cluster.

In [9]:
import subprocess

# Configuration
UPLOAD_TO_HPC = True  # Set to False to skip upload

if not UPLOAD_TO_HPC:
    print("⏭️  Skipping HPC upload (UPLOAD_TO_HPC=False)")
else:
    print("📤 Uploading manifests to HPC...\n")
    
    # Get all manifests to upload
    manifest_files = list(worker.manifests_dir.glob("cluster_*_manifest.json"))
    
    if not manifest_files:
        print("❌ No manifest files found to upload!")
    else:
        print(f"Found {len(manifest_files)} manifest files to upload")
        
        # Create remote manifests directory
        remote_manifests_dir = f"{config.HPC_ZARR_PATH}/manifests"
        
        print(f"Creating remote directory: {remote_manifests_dir}")
        mkdir_cmd = [
            'ssh',
            '-i', str(config.SSH_KEY),
            '-o', 'StrictHostKeyChecking=no',
            f"{config.HPC_USER}@{config.HPC_HOST}",
            f"mkdir -p {remote_manifests_dir}"
        ]
        
        result = subprocess.run(mkdir_cmd, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Failed to create remote directory: {result.stderr}")
        else:
            print("✓ Remote directory ready")
            
            # Upload all manifests in one SCP command
            print(f"\nUploading {len(manifest_files)} manifests...")
            
            remote_target = f"{config.HPC_USER}@{config.HPC_HOST}:{remote_manifests_dir}/"
            
            scp_cmd = [
                'scp',
                '-i', str(config.SSH_KEY),
                '-o', 'StrictHostKeyChecking=no',
                '-C',  # Enable compression
            ] + [str(f) for f in manifest_files] + [remote_target]
            
            result = subprocess.run(scp_cmd, capture_output=True, text=True)
            
            if result.returncode != 0:
                print(f"❌ Upload failed: {result.stderr}")
            else:
                print(f"✅ Successfully uploaded {len(manifest_files)} manifests to HPC")
                print(f"   Location: {remote_manifests_dir}")

📤 Uploading manifests to HPC...

Found 228 manifest files to upload
Creating remote directory: /scicore/home/meiera/schulz0022/projects/mining-net/data/landsat_mmap/manifests
✓ Remote directory ready

Uploading 228 manifests...
✅ Successfully uploaded 228 manifests to HPC
   Location: /scicore/home/meiera/schulz0022/projects/mining-net/data/landsat_mmap/manifests


## Step 6: Verify Upload (Optional)

Check that manifests exist on HPC.

In [10]:
# List manifests on HPC
remote_manifests_dir = f"{config.HPC_ZARR_PATH}/manifests"

print(f"Checking remote manifests at {remote_manifests_dir}...\n")

list_cmd = [
    'ssh',
    '-i', str(config.SSH_KEY),
    '-o', 'StrictHostKeyChecking=no',
    f"{config.HPC_USER}@{config.HPC_HOST}",
    f"ls -lh {remote_manifests_dir}/cluster_*_manifest.json | wc -l"
]

result = subprocess.run(list_cmd, capture_output=True, text=True)

if result.returncode == 0:
    remote_count = int(result.stdout.strip())
    local_count = len(manifest_files)
    
    print(f"📊 Manifest counts:")
    print(f"   Local:  {local_count}")
    print(f"   Remote: {remote_count}")
    
    if remote_count == local_count:
        print(f"\n✅ All manifests successfully uploaded!")
    else:
        print(f"\n⚠️  Mismatch detected. Difference: {abs(remote_count - local_count)}")
else:
    print(f"❌ Could not verify remote manifests: {result.stderr}")

Checking remote manifests at /scicore/home/meiera/schulz0022/projects/mining-net/data/landsat_mmap/manifests...

📊 Manifest counts:
   Local:  228
   Remote: 228

✅ All manifests successfully uploaded!


## Step 7: Summary Report

In [11]:
from src.data.manifest_reader import ManifestReader

reader = ManifestReader(worker.manifests_dir)
all_manifests = reader.list_all_manifests()

print(f"\n{'=' * 60}")
print(f"📋 Final Manifest Summary")
print(f"{'=' * 60}")
print(f"\nTotal manifests: {len(all_manifests)}")

# Group by country
by_country = defaultdict(list)
for m in all_manifests:
    by_country[m['country_code']].append(m)

print(f"\nBy country:")
for country, manifests in sorted(by_country.items()):
    total_tiles = sum(m['tile_count'] for m in manifests)
    print(f"   {country}: {len(manifests)} clusters, {total_tiles:,} tiles")

# Year coverage
all_years = set()
for m in all_manifests:
    all_years.update(m['years'])

print(f"\nYear coverage: {min(all_years)} - {max(all_years)} ({len(all_years)} years)")

# Total tiles
total_tiles = sum(m['tile_count'] for m in all_manifests)
print(f"Total tiles: {total_tiles:,}")

print(f"\n{'=' * 60}")
print("✅ Manifest generation and upload complete!")
print(f"{'=' * 60}")


📋 Final Manifest Summary

Total manifests: 228

By country:
   ZAF: 228 clusters, 43,103 tiles

Year coverage: 2019 - 2019 (1 years)
Total tiles: 43,103

✅ Manifest generation and upload complete!
